In [1]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import model_from_json
import numpy
import os
# fix random seed for reproducibility
numpy.random.seed(7)

# load pima indians dataset

import matplotlib.pyplot as plt
import pandas as pd
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
df = pd.read_csv('pima-indians-diabetes.data.csv', names=col_names)
# # load pima indians dataset
# dataset = numpy.loadtxt("pima-indians-diabetes.data.csv", delimiter=",")

# selection of relevant features
label = df['label']
df.drop('label', axis=1, inplace=True)

X, Y = df, label
df.describe()

# normalized data
df_norm = (df - df.mean()) / (df.max() - df.min())
df_norm.describe()
# X = df[['glucose','bmi']]

/home/yash/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-1.561251e-17,-2.312965e-18,3.035766e-18,1.098658e-17,-2.891206e-18,3.472338e-16,2.688821e-16,3.729655e-17
std,1.982105e-01,1.606664e-01,1.586542e-01,1.611335e-01,1.362222e-01,1.174987e-01,1.414725e-01,1.960039e-01
min,-2.261795e-01,-6.075102e-01,-5.664383e-01,-2.074390e-01,-9.432563e-02,-4.767895e-01,-1.681795e-01,-2.040148e-01
25%,-1.673560e-01,-1.100228e-01,-5.824155e-02,-2.074390e-01,-9.432563e-02,-6.993410e-02,-9.740662e-02,-1.540148e-01
50%,-4.970895e-02,-1.957051e-02,2.372567e-02,2.488426e-02,-5.827362e-02,1.106092e-04,-4.243224e-02,-7.068142e-02
75%,1.267616e-01,9.726366e-02,8.929944e-02,1.157934e-01,5.608809e-02,6.866501e-02,6.591533e-02,1.293186e-01
max,7.738205e-01,3.924898e-01,4.335617e-01,7.925610e-01,9.056744e-01,5.232105e-01,8.318205e-01,7.959852e-01


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=66)


In [4]:
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
rfc_cv_score = cross_val_score(rfc, X, Y, cv=10, scoring='roc_auc')

In [15]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[139  18]
 [ 33  41]]


=== Classification Report ===
             precision    recall  f1-score   support

          0       0.81      0.89      0.84       157
          1       0.69      0.55      0.62        74

avg / total       0.77      0.78      0.77       231



=== All AUC Scores ===
[0.78333333 0.83       0.82407407 0.73185185 0.81222222 0.86814815
 0.87148148 0.90444444 0.81076923 0.84846154]


=== Mean AUC Score ===
('Mean AUC Score - Random Forest: ', 0.8284786324786324)


In [17]:
'''
We’ll use RandomizedSearchCV from sklearn to optimize our hyperparamaters. 
Koehrsen uses a full grid of hyperparameters in his article, but I found that this could take a very substantial time to run in practice.
I decided to focus on 3 hyperparameters: n_estimators, max_features, and max_depth.
'''
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=1000, max_features=auto, max_depth=140 .............
[CV] n_estimators=1400, max_features=auto, max_depth=100 .............
[CV] n_estimators=1000, max_features=auto, max_depth=140 .............
[CV] n_estimators=1000, max_features=auto, max_depth=140 .............
[CV] n_estimators=1400, max_features=auto, max_depth=100 .............
[CV] n_estimators=800, max_features=sqrt, max_depth=260 ..............
[CV] n_estimators=1400, max_features=auto, max_depth=100 .............
[CV] n_estimators=800, max_features=sqrt, max_depth=260 ..............
[CV]  n_estimators=800, max_features=sqrt, max_depth=260, total=   2.5s
[CV] n_estimators=800, max_features=sqrt, max_depth=260 ..............
[CV]  n_estimators=800, max_features=sqrt, max_depth=260, total=   2.5s
[CV] n_estimators=2000, max_features=auto, max_depth=300 .............
[CV]  n_estimators=1000, max_features=auto, max_depth=140, total=   3.1s
[CV] n_est

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   13.7s


[CV]  n_estimators=1400, max_features=auto, max_depth=260, total=   4.8s
[CV] n_estimators=1400, max_features=auto, max_depth=500 .............
[CV]  n_estimators=800, max_features=auto, max_depth=380, total=   2.7s
[CV] n_estimators=1400, max_features=auto, max_depth=500 .............
[CV]  n_estimators=2000, max_features=auto, max_depth=100, total=   6.9s
[CV] n_estimators=1400, max_features=auto, max_depth=500 .............
[CV]  n_estimators=2000, max_features=auto, max_depth=100, total=   6.8s
[CV] n_estimators=1000, max_features=sqrt, max_depth=300 .............
[CV]  n_estimators=2000, max_features=auto, max_depth=100, total=   6.7s
[CV] n_estimators=1000, max_features=sqrt, max_depth=300 .............
[CV]  n_estimators=1200, max_features=auto, max_depth=180, total=   4.2s
[CV] n_estimators=1000, max_features=sqrt, max_depth=300 .............
[CV]  n_estimators=1200, max_features=auto, max_depth=180, total=   4.1s
[CV] n_estimators=1800, max_features=auto, max_depth=420 .......

[CV] n_estimators=1400, max_features=sqrt, max_depth=260 .............
[CV]  n_estimators=1400, max_features=sqrt, max_depth=420, total=   5.3s
[CV] n_estimators=1400, max_features=sqrt, max_depth=260 .............
[CV]  n_estimators=800, max_features=sqrt, max_depth=380, total=   3.0s
[CV] n_estimators=1400, max_features=sqrt, max_depth=260 .............
[CV]  n_estimators=800, max_features=sqrt, max_depth=380, total=   2.8s
[CV] n_estimators=1800, max_features=auto, max_depth=None ............
[CV]  n_estimators=800, max_features=sqrt, max_depth=380, total=   3.1s
[CV] n_estimators=1800, max_features=auto, max_depth=None ............
[CV]  n_estimators=1600, max_features=sqrt, max_depth=None, total=   5.6s
[CV] n_estimators=1800, max_features=auto, max_depth=None ............
[CV]  n_estimators=1600, max_features=sqrt, max_depth=None, total=   5.6s
[CV] n_estimators=1200, max_features=auto, max_depth=140 .............
[CV]  n_estimators=1600, max_features=sqrt, max_depth=None, total=

[CV] n_estimators=200, max_features=auto, max_depth=220 ..............
[CV]  n_estimators=1400, max_features=auto, max_depth=220, total=   4.9s
[CV] n_estimators=200, max_features=auto, max_depth=220 ..............
[CV]  n_estimators=200, max_features=auto, max_depth=220, total=   0.7s
[CV] n_estimators=200, max_features=auto, max_depth=220 ..............
[CV]  n_estimators=200, max_features=auto, max_depth=220, total=   0.7s
[CV] n_estimators=1000, max_features=auto, max_depth=500 .............
[CV]  n_estimators=200, max_features=auto, max_depth=220, total=   0.7s
[CV] n_estimators=1000, max_features=auto, max_depth=500 .............
[CV]  n_estimators=1800, max_features=sqrt, max_depth=140, total=   6.0s
[CV] n_estimators=1000, max_features=auto, max_depth=500 .............
[CV]  n_estimators=1800, max_features=sqrt, max_depth=140, total=   6.1s
[CV] n_estimators=1400, max_features=auto, max_depth=460 .............


...

[CV]  n_estimators=1800, max_features=sqrt, max_depth=140, total=   6.1s
[CV] n_estimators=1400, max_features=auto, max_depth=460 .............
[CV]  n_estimators=1000, max_features=auto, max_depth=500, total=   3.5s
[CV] n_estimators=1400, max_features=auto, max_depth=460 .............
[CV]  n_estimators=2000, max_features=sqrt, max_depth=340, total=   6.9s
[CV] n_estimators=1600, max_features=auto, max_depth=500 .............
[CV]  n_estimators=1000, max_features=auto, max_depth=500, total=   3.6s
[CV] n_estimators=1600, max_features=auto, max_depth=500 .............
[CV]  n_estimators=1000, max_features=auto, max_depth=500, total=   3.5s
[CV] n_estimators=1600, max_features=auto, max_depth=500 .............
[CV]  n_estimators=2000, max_features=sqrt, max_depth=340, total=   7.1s
[CV] n_estimators=800, max_features=sqrt, max_depth=460 ..............
[CV]  n_estimators=2000, max_features=sqrt, max_depth=340, total=   7.3s
[CV] n_estimators=800, max_features=sqrt, max_depth=460 .......

[CV]  n_estimators=1800, max_features=auto, max_depth=300, total=   6.2s
[CV] n_estimators=1600, max_features=sqrt, max_depth=420 .............
[CV]  n_estimators=1000, max_features=sqrt, max_depth=460, total=   3.4s
[CV] n_estimators=1600, max_features=sqrt, max_depth=420 .............
[CV]  n_estimators=1000, max_features=sqrt, max_depth=460, total=   3.6s
[CV] n_estimators=400, max_features=sqrt, max_depth=300 ..............
[CV]  n_estimators=1000, max_features=sqrt, max_depth=460, total=   3.4s
[CV] n_estimators=400, max_features=sqrt, max_depth=300 ..............
[CV]  n_estimators=400, max_features=sqrt, max_depth=300, total=   1.4s
[CV]  n_estimators=1600, max_features=sqrt, max_depth=300, total=   5.8s
[CV] n_estimators=2000, max_features=auto, max_depth=140 .............
[CV] n_estimators=400, max_features=sqrt, max_depth=300 ..............
[CV]  n_estimators=1600, max_features=sqrt, max_depth=300, total=   5.4s
[CV] n_estimators=2000, max_features=auto, max_depth=140 .......

[CV] n_estimators=600, max_features=auto, max_depth=180 ..............
[CV]  n_estimators=1200, max_features=sqrt, max_depth=140, total=   4.3s
[CV] n_estimators=600, max_features=auto, max_depth=180 ..............
[CV]  n_estimators=600, max_features=auto, max_depth=180, total=   2.1s
[CV] n_estimators=800, max_features=auto, max_depth=None .............
[CV]  n_estimators=1200, max_features=sqrt, max_depth=140, total=   4.3s
[CV] n_estimators=800, max_features=auto, max_depth=None .............
[CV]  n_estimators=600, max_features=auto, max_depth=180, total=   2.1s
[CV] n_estimators=800, max_features=auto, max_depth=None .............
[CV]  n_estimators=1200, max_features=sqrt, max_depth=140, total=   4.4s
[CV] n_estimators=1800, max_features=auto, max_depth=380 .............
[CV]  n_estimators=600, max_features=auto, max_depth=180, total=   2.1s
[CV] n_estimators=1800, max_features=auto, max_depth=380 .............
[CV]  n_estimators=800, max_features=auto, max_depth=None, total=   

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished


{'n_estimators': 1400, 'max_features': 'sqrt', 'max_depth': 180}


In [20]:
rfc = RandomForestClassifier(n_estimators=1400, max_depth=180, max_features='sqrt')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, Y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[140  17]
 [ 34  40]]


=== Classification Report ===
             precision    recall  f1-score   support

          0       0.80      0.89      0.85       157
          1       0.70      0.54      0.61        74

avg / total       0.77      0.78      0.77       231



=== All AUC Scores ===
[0.77777778 0.83518519 0.83592593 0.73037037 0.81814815 0.85777778
 0.87185185 0.90777778 0.805      0.86384615]


=== Mean AUC Score ===
('Mean AUC Score - Random Forest: ', 0.8303660968660969)
